In [102]:
import pandas as pd
import numpy as np
from numpy import set_printoptions
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import confusion_matrix, f1_score, accuracy_score
from sklearn.preprocessing import Normalizer 
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import OrdinalEncoder 
from sklearn.metrics import classification_report
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score


In [55]:
advert_data = pd.read_csv('C:\\allAccountsSending\\github projects\\predict_user_clicked_ad.csv')

In [56]:
advert_data.head()


,Daily Time Spent on Site,Age,Area Income,Daily Internet Usage,Ad Topic Line,City,Male,Country,Timestamp,Clicked on Ad
0,68.95,35,61833.90,256.09,Cloned 5thgeneration orchestration,Wrightburgh,0,Tunisia,2016-03-27 00:53:11,0
1,80.23,31,68441.85,193.77,Monitored national standardization,West Jodi,1,Nauru,2016-04-04 01:39:02,0
2,69.47,26,59785.94,236.50,Organic bottom-line service-desk,Davidton,0,San Marino,2016-03-13 20:35:42,0
3,74.15,29,54806.18,245.89,Triple-buffered reciprocal time-frame,West Terrifurt,1,Italy,2016-01-10 02:31:19,0
4,68.37,35,73889.99,225.58,Robust logistical utilization,South Manuel,0,Iceland,2016-06-03 03:36:18,0


In [58]:
advert_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 10 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Daily Time Spent on Site  1000 non-null   float64
 1   Age                       1000 non-null   int64  
 2   Area Income               1000 non-null   float64
 3   Daily Internet Usage      1000 non-null   float64
 4   Ad Topic Line             1000 non-null   object 
 5   City                      1000 non-null   object 
 6   Male                      1000 non-null   int64  
 7   Country                   1000 non-null   object 
 8   Timestamp                 1000 non-null   object 
 9   Clicked on Ad             1000 non-null   int64  
dtypes: float64(3), int64(3), object(4)
memory usage: 78.2+ KB


In [115]:
advert_data['Clicked on Ad'].value_counts() # 

0    500
1    500
Name: Clicked on Ad, dtype: int64

In [59]:
advert_data = advert_data.astype({'Timestamp':'string'})
advert_data['Year'] = advert_data['Timestamp'].str.slice(0,4)
advert_data['Month'] = advert_data['Timestamp'].str.slice(6,7)
advert_data = advert_data.drop('Timestamp', axis = 1)



In [61]:
advert_data['Year'] = pd.to_numeric(advert_data['Year'])
advert_data['Month'] = pd.to_numeric(advert_data['Month'])

In [93]:
# encode categorical variables to numeric using the ordinal encoder
feature = ['Ad Topic Line' , 'City', 'Country',]
encode = OrdinalEncoder()
advert_data[feature] = encode.fit_transform(advert_data[feature])



,Daily Time Spent on Site,Age,Area Income,Daily Internet Usage,Ad Topic Line,City,Male,Country,Clicked on Ad,Year,Month
count,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.00000,1000.00000,1000.0,1000.000000
mean,65.000200,36.009000,55000.000080,180.000100,499.500000,486.323000,0.481000,115.41000,0.50000,2016.0,3.817000
std,15.853615,8.785562,13414.634022,43.902339,288.819436,279.311571,0.499889,69.94372,0.50025,0.0,1.926973
min,32.600000,19.000000,13996.500000,104.780000,0.000000,0.000000,0.000000,0.00000,0.00000,2016.0,1.000000
25%,51.360000,29.000000,47031.802500,138.830000,249.750000,246.750000,0.000000,54.00000,0.00000,2016.0,2.000000
50%,68.215000,35.000000,57012.300000,183.130000,499.500000,484.500000,0.000000,113.50000,0.50000,2016.0,4.000000
75%,78.547500,42.000000,65470.635000,218.792500,749.250000,727.250000,1.000000,177.00000,1.00000,2016.0,5.000000
max,91.430000,61.000000,79484.800000,269.960000,999.000000,968.000000,1.000000,236.00000,1.00000,2016.0,7.000000


In [94]:
# Split the data set intpo the independent and dependent variables

# drop the encoded features so that only the features with initial numerical values will be rescaled and normalized 
input_data = advert_data.drop(['Clicked on Ad','Month','Year','Country','Ad Topic Line','City'],axis=1)
print(input_data.describe())

# save the dropped features in a dataframe
dropped_input_data = advert_data.loc[:,['Month','Year','Country','Ad Topic Line','City']]

# the target variable 
target_dataY = advert_data['Clicked on Ad']

advert_input = input_data.values
rescaled_input = MinMaxScaler(feature_range = (0,1)).fit_transform(advert_input)
normalized_input = Normalizer().fit(rescaled_input).transform(rescaled_input)
set_printoptions(precision=3)




normalized_input = pd.DataFrame(normalized_input, columns = input_data.columns)

# concatenate the independent dataset 
processed_inputX = pd.concat([normalized_input, dropped_input_data], axis = 1)

processed_inputX


       Daily Time Spent on Site          Age   Area Income  \
count               1000.000000  1000.000000   1000.000000   
mean                  65.000200    36.009000  55000.000080   
std                   15.853615     8.785562  13414.634022   
min                   32.600000    19.000000  13996.500000   
25%                   51.360000    29.000000  47031.802500   
50%                   68.215000    35.000000  57012.300000   
75%                   78.547500    42.000000  65470.635000   
max                   91.430000    61.000000  79484.800000   

       Daily Internet Usage         Male  
count           1000.000000  1000.000000  
mean             180.000100     0.481000  
std               43.902339     0.499889  
min              104.780000     0.000000  
25%              138.830000     0.000000  
50%              183.130000     0.000000  
75%              218.792500     1.000000  
max              269.960000     1.000000  


,Daily Time Spent on Site,Age,Area Income,Daily Internet Usage,Male,Month,Year,Country,Ad Topic Line,City
0,0.448305,0.276401,0.529995,0.664628,0.000000,3,2016,215.0,91.0,961.0
1,0.491036,0.173286,0.504230,0.326750,0.606501,4,2016,147.0,464.0,903.0
2,0.504153,0.134072,0.562458,0.641479,0.000000,3,2016,184.0,566.0,111.0
3,0.431938,0.145613,0.381108,0.522456,0.611574,1,2016,103.0,903.0,939.0
4,0.442731,0.277390,0.665940,0.532511,0.000000,6,2016,96.0,766.0,805.0
...,...,...,...,...,...,...,...,...,...,...
995,0.417439,0.159322,0.533079,0.382273,0.608322,2,2016,116.0,345.0,126.0
996,0.214745,0.418219,0.554859,0.121227,0.675584,4,2016,26.0,359.0,488.0
997,0.235770,0.555327,0.316298,0.068792,0.728867,2,2016,140.0,263.0,798.0
998,0.508945,0.000000,0.556295,0.656895,0.000000,3,2016,85.0,641.0,935.0


In [82]:
X_train, X_test, y_train, y_test = train_test_split(processed_inputX, target_dataY, test_size=0.3, random_state=101)

In [83]:
lorclf = LogisticRegression()

In [85]:
lorclf.fit(processed_inputX,target_dataY)


C:\Users\anaconda\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [87]:
predictions = lorclf.predict(X_test)

In [89]:
print(classification_report(y_test,predictions))

              precision    recall  f1-score   support

           0       0.91      0.97      0.94       157
           1       0.97      0.89      0.93       143

    accuracy                           0.93       300
   macro avg       0.94      0.93      0.93       300
weighted avg       0.94      0.93      0.93       300



In [98]:
print(f'Confusion matrix{confusion_matrix(y_test, predictions)}')
print(f'F1 score : {f1_score(y_test, predictions)}')
print(f'accuracy: {accuracy_score(y_test, predictions)}')

Confusion matrix[[153   4]
 [ 16 127]]
F1 score : 0.927007299270073
accuracy: 0.9333333333333333


In [112]:
numFolds = 10
kfold = KFold(n_splits = numFolds)# random_state = 7)
cv_results = cross_val_score(lorclf, processed_inputX, target_dataY, cv=kfold, scoring = 'accuracy')
msg = (cv_results.mean(), cv_results.std()) #cv_reslts.mean()-accuracyn score and cv_results.std -error
print(msg)

C:\Users\anaconda\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\anaconda\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n

(0.9179999999999999, 0.03091924966748061)


C:\Users\anaconda\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\anaconda\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n